### Reescrever snapshot hdf5

Substituindo partículas de gás por partículas de disco

In [1]:
import h5py
import numpy as np

In [2]:
#snapshot a ser copiado
file = h5py.File('snapshot_000.hdf5', 'r+')
file.keys()

<KeysViewHDF5 ['Config', 'Header', 'Parameters', 'PartType0', 'PartType1', 'PartType2', 'PartType3', 'PartType4', 'PartType5']>

In [3]:
#ver número de cada tipo de patícula
print(file['Header'].attrs['NumPart_ThisFile'])
print(file['Header'].attrs['NumPart_Total'])

[  30715 1000000  160000   40000  169285    6500]
[  30715 1000000  160000   40000  169285    6500]


In [4]:
160000+30715

190715

In [5]:
#alterar número e substituir
#reescreve o original!
numpart_file = np.array([0, 1000000, 190715, 40000, 169285, 6500], dtype=np.uint64)
file['Header'].attrs['NumPart_ThisFile'] = numpart_file 
file['Header'].attrs['NumPart_Total'] = numpart_file 

In [6]:
#verificação
print(file['Header'].attrs['NumPart_ThisFile'])
print(file['Header'].attrs['NumPart_Total'])

[      0 1000000  190715   40000  169285    6500]
[      0 1000000  190715   40000  169285    6500]


In [7]:
f_gas = file['PartType0']
f_halo = file['PartType1']
f_disk = file['PartType2']
f_bulge = file['PartType3']
f_star = file['PartType4']
f_bndry = file['PartType5']

In [8]:
# #disk
# x_disk = np.array(f_disk['Coordinates'][:,0])
# y_disk = np.array(f_disk['Coordinates'][:,1])
# z_disk = np.array(f_disk['Coordinates'][:,2])
# vx_disk = np.array(f_disk['Velocities'][:,0])
# vy_disk = np.array(f_disk['Velocities'][:,1])
# vz_disk = np.array(f_disk['Velocities'][:,2])
# m_disk = np.array(f_disk['Masses'])
# n_disk = np.array(f_disk['ParticleIDs'])

# x_gas = np.array(f_gas['Coordinates'][:,0])
# y_gas = np.array(f_gas['Coordinates'][:,1])
# z_gas = np.array(f_gas['Coordinates'][:,2])
# vx_gas = np.array(f_gas['Velocities'][:,0])
# vy_gas = np.array(f_gas['Velocities'][:,1])
# vz_gas = np.array(f_gas['Velocities'][:,2])
# m_gas = np.array(f_gas['Masses'])
# n_gas = np.array(f_gas['ParticleIDs'])

In [9]:
#concatenar gas e disk
x = np.concatenate((f_disk['Coordinates'][:,0], f_gas['Coordinates'][:,0]))
y = np.concatenate((f_disk['Coordinates'][:,1], f_gas['Coordinates'][:,1]))
z = np.concatenate((f_disk['Coordinates'][:,2], f_gas['Coordinates'][:,2]))

vx = np.concatenate((f_disk['Velocities'][:,0], f_gas['Velocities'][:,0]))
vy = np.concatenate((f_disk['Velocities'][:,1], f_gas['Velocities'][:,1]))
vz = np.concatenate((f_disk['Velocities'][:,2], f_gas['Velocities'][:,2]))

m = np.concatenate((f_disk['Masses'], f_gas['Masses']))
n = np.concatenate((f_disk['ParticleIDs'], f_gas['ParticleIDs']))

In [10]:
#escrever novo arquivo
file2 = h5py.File('teste3.hdf5', 'w')

In [11]:
#copiar grupos que farão parte do novo arquivo, ignorando os que não serão necessários
file.copy('Config', file2)
file.copy('Header', file2)
file.copy('Parameters', file2)
file.copy('PartType1', file2)
file.copy('PartType3', file2)
file.copy('PartType4', file2)
file.copy('PartType5', file2)

In [12]:
#criar grupo disk
file2.create_group('PartType2')

<HDF5 group "/PartType2" (0 members)>

In [15]:
file2['PartType2'].keys()

<KeysViewHDF5 ['Coordinates', 'Masses', 'ParticleIDs', 'Velocities']>

In [14]:
#criar datasets para o grupo disk
file2['PartType2'].create_dataset('ParticleIDs', data=n)
file2['PartType2'].create_dataset('Masses', data=m)
file2['PartType2'].create_dataset('Coordinates', data=np.array([x,y,z]).T)
file2['PartType2'].create_dataset('Velocities', data=np.array([vx,vy,vz]).T)

<HDF5 dataset "Velocities": shape (190715, 3), type "<f4">

In [18]:
#verificação
print(file2.keys())
print(np.array(file2['PartType2']['Velocities']))

<KeysViewHDF5 ['Config', 'Header', 'Parameters', 'PartType1', 'PartType2', 'PartType3', 'PartType4', 'PartType5']>
[[ -76.38782   -157.6146      -4.909666 ]
 [ -74.407585  -115.570496   -23.145695 ]
 [-182.18864    -75.20026    -20.134573 ]
 ...
 [ 142.23245     -6.954377    -4.665039 ]
 [  94.89494     56.318153     2.7505937]
 [ 107.246056   -38.65506      7.9782057]]


In [19]:
file.close()

In [20]:
file2.close()